### Exploratory Data Analysis

Для того чтобы провести разведочный анализ данных необходимо их загрузить. Данные хранятся в сериализованном формате pickle на S3, загрузим их и десериализуем с помощью pickle.

In [6]:
# загружаем данные
!wget https://storage.yandexcloud.net/habr-hse/datasets/2023/articles.pkl

--2023-11-14 12:44:41--  https://storage.yandexcloud.net/habr-hse/datasets/2023/articles.pkl
Resolving storage.yandexcloud.net (storage.yandexcloud.net)... 213.180.193.243, 2a02:6b8::1d9
Connecting to storage.yandexcloud.net (storage.yandexcloud.net)|213.180.193.243|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 310615187 (296M) [application/octet-stream]
Saving to: ‘articles.pkl’

articles.pkl        100%[===================>] 296.23M  56.9MB/s    in 4.9s    

2023-11-14 12:44:46 (60.4 MB/s) - ‘articles.pkl’ saved [310615187/310615187]



In [1]:
import pickle
import matplotlib.pyplot as plt

Теперь откроем загруженный файл на чтение в бинарном формате и десериализуем его в Python-объект(pandas.DataFrame) используя функцию pickle.loads.

In [3]:
with open('./articles.pkl', 'rb') as f:
    data = pickle.loads(f.read())

data.sample(5)

,id,title,text,time,hubs,tags
8798,746794,How to make a robot? What is first,"I develop robots and I'm often asked - ""how to...",8 июл в 23:16,JavaScript *|Node.JS *|Разработка робототехник...,robotics|raspberrypi|raspberry pi|nodejs
7427,750116,С декабря 2023 года в рунете планируется огран...,С декабря 2023 года в рунете планируется добро...,25 июл в 08:12,Информационная безопасность *|Веб-разработка *...,рунет|аккаунт|российскией сервис|регистрация
2594,761600,Где в Москве жить «неплохо»,"В одной прошлых публикации про ""где бы вы точн...",21 сен в 08:00,PostgreSQL *|OpenStreetMap *|Геоинформационные...,где жить неплохо|москва|мкад|экология|тишина
5487,754700,СПб ФИЦ РАН создали промышленный микрокомпьюте...,В Санкт‑Петербургском Федеральном исследовател...,15 авг в 12:57,Производство и разработка электроники *|IT-ком...,спб фиц ран|промышленный компьютер|микрокомпью...
10013,743798,Zod: Типизация и валидация Вашего .env (Vite +...,ВведениеНередко в проектах необходимо заводить...,30 июн в 12:01,JavaScript *|ReactJS *|TypeScript *,typescript|vite|environment variables|config|z...


Изучим структуру собранных данных, для этого посмотрим какие столбцы есть в датафрейме и какие типы данных в них хранятся.

In [23]:
data.dtypes

id        int64
title    object
text     object
time     object
hubs     object
tags     object
dtype: object

Данные содержат следующие столбцы:
- id: уникальный идентификатор статьи
- title: заголовок статьи
- text: текст статьи
- time: время публикации
- hubs: Хабы(Сообщества) с которыми связана статья
- tags: Теги статьи

Проверим есть ли в данных пропуски, начнем со столбца "title"

In [43]:
data['title'].map(lambda x: len(x)).value_counts().sort_index()

title
2        1
3        1
4        2
6        1
7        2
      ... 
117    107
118    121
119    113
120    114
121      1
Name: count, Length: 119, dtype: int64

В данных есть две статьи с аномальными заголовками -- одна с очень большим (121 символ) и вторая с маленьким -- (2) символа, посмотрим на них

In [44]:
data[data['title'].map(lambda x: len(x) == 2 or len(x) == 121)]

,id,title,text,time,hubs,tags
14752,732626,96,Бриллианты. Куда ни глянь – всюду в свете фон...,1 мая в 12:52,Научная фантастика,читальный зал|досуг|чтиво|самодеятельность
17842,725472,Создание сервера для онлайн ММО игр на PHP и U...,В этой серии статей речь пойдет о том что тако...,11 июн в 17:38,Разработка игр *|Сетевые технологии *|C# *|Uni...,php|unity|интерполяция|экстраполяция|ping|webs...


Видим, что обе статьи не являются выбросами.

Аналогично изучим другие столбцы

Смотрим на длину текстов в символах

In [9]:
data['text'].map(lambda x: len(x)).value_counts().sort_index()

text
197       1
231       1
239       1
259       1
260       1
         ..
159696    1
176555    1
216898    1
233741    1
294738    1
Name: count, Length: 11960, dtype: int64

Пустые тексты отсутствуют

In [11]:
data['tags'].map(lambda x: len(x.split('|'))).value_counts().sort_index()

tags
1      425
2     1279
3     2681
4     3268
5     2938
6     2442
7     1854
8     1537
9     1079
10    2011
11      86
12     123
13      63
14      51
15      45
16      26
17      19
18      14
19       9
20      10
21       8
22       7
23       5
24       5
25       2
26       1
27       2
28       2
29       2
30       1
31       3
33       2
Name: count, dtype: int64

Для каждой статьи есть 1 или больше тегов, статей без тегов нет

In [8]:
data['hubs'].map(lambda x: len(x.split('|'))).value_counts().sort_index()

hubs
1    1645
2    4930
3    5597
4    4073
5    3751
6       4
Name: count, dtype: int64

Каждая статья принадлежит ненулевому количеству хабов